[healpix]: https://healpix.jpl.nasa.gov
[swift]: http://swift.asdc.asi.it/
[swift_deepsky]: https://github.com/chbrandt/swift_deepsky


# Sky discretization with HEALpix

The task we want to accomplish is that of covering a given region of the sky with circles. Not a new problem to the human existence, I want to do this exercise using [Healpix][healpix].

Healpix will be used here for binning a series of coordinates; the coordinates may be a real list of sources or an artifical one.
In any case, the binning will downsample the list of coordinates.

An application for the downsampling is the definition of observational pointings on a survey, where we have to define the minimum amount of to cover a specific region.

The pointings are defined by a central coordinate -- RA,Dec -- and a radius value defining the size of the circle (i.e, field of view) each pointing covers.


## Swift/XRT

At the end of the day we want to define the coverage of a region as if done by the [Swift/XRT][swift] telescope.
Such list of pointings will then be used by our [Swift-DeepSky][swift_deepsky] pipeline.

The Swift' field-of-view is of $12\ arcmin$.

In [15]:
def naive_region2pointings(ramin, ramax, decmin, decmax, radius):
    from astropy import units

    bbox = dict(ramin=ramin * units.degree,
                ramax=ramax * units.degree,
                decmin=decmin * units.degree,
                decmax=decmax * units.degree)

    try:
        rad = radius.to('arcmin')
    except:
        rad = radius * units.arcmin

    # get the closest smaller size of healpix elements
    from moc import utils
    level = utils.size_to_level(rad, truncate=True)
    #
    from moc import core
    dsize = core.HEALPIX_LEVELS[level]

    # create a grid of (fake) coordinates, to then create a MOC table from it
    step_size = dsize.to('deg').value/2**0.5

    import numpy as np
    ra_vec = np.arange(bbox['ramin'].value, bbox['ramax'].value, step_size)
    dec_vec = np.arange(bbox['decmin'].value, bbox['decmax'].value, step_size)
    
    import itertools
    grid = np.asarray(list(itertools.product(ra_vec,dec_vec))).T
    
    return grid

In [16]:
from astropy import units
radius = 12 * units.arcmin
grid = naive_region2pointings(21, 23, 32, 34, radius)

ra_vec,dec_vec = grid

In [17]:
# from astropy.coordinates import SkyCoord
# coords = SkyCoord(grid,unit='deg')

## A list of fixed-order Healpix elements

In [18]:
def coords2hpix(radec, level):
    """
    Input:
     - radec : array shape = (N,2)
     - level : healpix level representing the size of each element
    """
    from healpy import pixelfunc
    
    nside = 2**level

    pix_to_visit = []
    for ra,dec in radec:
        ipix = pixelfunc.ang2pix(nside, ra, dec, nest=True, lonlat=True)
        pix_to_visit.append(ipix)
        
    return pix_to_visit


def hpix2coords(hpixs, level):
    """
    Transform healpix elements to coordinates
    
    Input:
     - hpixs : list of healpix elements
     - level : healpix level to consider
    """
    from healpy import pixelfunc

    nside = 2**level

    coords_to_visit = []
    for ipix in hpixs:
        c = pixelfunc.pix2ang(nside, ipix, nest=True, lonlat=True)
        coords_to_visit.append(c)
    
    return coords_to_visit


def coords_binning_hpix(radec, level, unique=False):
    """
    Return the healpix binning of 'ra,dec' at some 'level'
    
    Input:
     - radec : array shape = (N,2)
     - level : healpix level representing the size of each element
     - unique: if True, remove (binned) duplicates
    """
    
    pix_to_visit = coords2hpix(radec, level)
    
    if unique:
        pix_to_visit = set(pix_to_visit)
        
    return hpix2coords(pix_to_visit, level)
    

def coords_binning(ra, dec, rad, unique=False, truncate=False):
    """
    Return list of binned ra,dec in closest healpix size to 'rad'
    
    Input:
     - ra       : list of (degree) RA coordinates
     - dec      : list of (degree) Dec coordinates
     - rad      : radius/size value to bin;
                  the actual value will be defined by one of Healpix levels;
                  the closest level may be above or below, see 'truncate'
     - unique   : if True, remove duplicated binned coordinates;
                  if False, returned list matches input ra,dec
     - truncate : if True, use the larger-closest healpix element size;
                  if False, use the smaller-closest healpix element
    """
    assert len(ra) == len(dec)
    assert rad > 0
    
    import itertools
    grid = list(itertools.product(ra, dec))
    
    from moc import utils
    level = utils.size_to_level(rad, truncate=truncate)
    
    coords_to_visit = coords_binning_hpix(grid, level, unique=unique)
    
    return coords_to_visit,level

        ## We don't need the neighbours, but if that was the case:
        #
        #nipixs = pixelfunc.get_all_neighbours(nside, c[0], c[1], nest=True, lonlat=True)
        #cns = []
        #for nipix in nipixs:
        #    cns.append(pixelfunc.pix2ang(nside, nipix, nest=True, lonlat=True))
        #coords_to_visit.extend(cns)
        

In [19]:
coords_to_visit,level = coords_binning(ra_vec, dec_vec, radius, True, True)

In [23]:
from moc import core
x_hp,y_hp = list(zip(*coords_to_visit))
r_hp = core.HEALPIX_LEVELS[level].to('deg').value
r_og = radius.to('deg').value

r_middle = (r_hp + r_og)/2

print('radii:', r_hp, r_og, r_middle)

radii: 0.229 0.2 0.21450000000000002


In [24]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

fig = figure()
fig.circle(x_hp,y_hp,radius=r_og, alpha=0.1, color='red')
fig.circle(x_hp,y_hp,radius=r_hp, alpha=0.1)
fig.circle(x_hp,y_hp,radius=r_middle, fill_alpha=0, line_color='black')
show(fig)

Loading BokehJS ...

## A real catalog

Let's build the pointings visualization from a real list of RA,Dec

We'll take the LaMassa catalog, in the file `lamassa_chandracsv`

In [2]:
import pandas

lamassa = pandas.read_csv('lamassa_chandra.csv')
lamassa.describe()

,RAdeg,DEdeg
count,1146.000000,1146.000000
mean,140.746483,0.000745
std,153.277262,0.661634
min,0.508000,-1.456000
25%,19.782000,-0.437750
50%,40.538000,0.191000
75%,334.452750,0.443500
max,359.706000,1.485000


In [5]:
from astropy import units
ra_vec = lamassa['RAdeg']
dec_vec = lamassa['DEdeg']
radius = 12 * units.arcmin

coords_to_visit,level = coords_binning(ra_vec, dec_vec, radius, True, True)

In [11]:
x,y = list(zip(*coords_to_visit))
r = radius.to('deg').value
print(len(x))

1895


In [10]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

fig = figure()
fig.circle(x, y, radius=r, fill_alpha=0.1)
show(fig)

Loading BokehJS ...

## Create the list of pointings

In [12]:
import pandas

df = pandas.DataFrame({'ra':x, 'dec':y, 'radius':radius.to('arcmin').value})
df

,dec,ra,radius
0,0.149208,45.000000,12.0
1,0.298417,45.175781,12.0
2,0.298417,359.824219,12.0
3,0.447628,45.000000,12.0
4,0.447628,0.351562,12.0
5,0.596842,0.527344,12.0
6,0.447628,45.351562,12.0
7,0.746060,0.351562,12.0
8,0.447628,359.648438,12.0
9,0.596842,359.824219,12.0


In [ ]:
# df = df.sample(frac=1).reset_index(drop=True)

In [13]:
def sds_cmdline(df_row):
    cmdline = 'docker run --rm -v /scratch/work:/work chbrandt/swift_deepsky'
    ra = df_row['ra']
    dec = df_row['dec']
    radius = df_row['radius']
    lbl = '{:d}_{:d}_{:d}'.format(int(df_row['index']), int(ra), int(dec))
    cmdline += ' --ra {:.5f} --dec {:.5f} --radius {:.3f} --label {!s}'
    cmdline = cmdline.format(ra, dec, radius, lbl)
#     print(cmdline)
    return cmdline
    
cmdlines = df.reset_index().apply(sds_cmdline, axis=1)

In [14]:
cmdlines.to_csv('swift_deepsky_pointings_list.txt', header=False, index=False)

In [ ]:
%cat swift_deepsky_pointings_list.txt